In [ ]:
## MSCI China + Financials
In this script, we will be backtesting on the monthly dataset. Testing out the performance metrics of the AI/ML financial model on individual financial firms, to faciliate decision making process of the buying or holding

import datetime
from dateutil.relativedelta import relativedelta

import warnings
warnings.filterwarnings('ignore')

%run import_library.ipynb

# msci china financial price

msci_financial_df = pd.read_excel('inputs/MSCI_CHINA_Financial_PriceHistory.xlsx', skiprows=1)

msci_financial_df = standardize_col_names(msci_financial_df)
msci_financial_df = msci_financial_df[['date', 'price']]
msci_financial_df.rename(columns={'price': f'msci_financial_price'}, inplace=True)
msci_financial_df['date'] = pd.to_datetime(msci_financial_df.date, format='%d/%m/%Y') # set the date based on yymmdd
msci_financial_df = msci_financial_df.set_index('date')
msci_financial_df.dropna(inplace=True)

msci_financial_df.sort_index(ascending=True, inplace=True)
msci_financial_df = aggregate_period(msci_financial_df, 'M', 'last')
msci_financial_df = msci_financial_df.apply(pd.to_numeric)
msci_financial_df['msci_financial_returns'] = msci_financial_df['msci_financial_price']/msci_financial_df['msci_financial_price'].shift(1) - 1
msci_financial_df['msci_financial_returns_lead_1'] = msci_financial_df['msci_financial_returns'].shift(-1)
msci_financial_df

# msci china price

msci_china_df = pd.read_excel('inputs/MSCI_CHINA_PriceHistory.xlsx', skiprows=1)

msci_china_df = standardize_col_names(msci_china_df)
msci_china_df = msci_china_df[['date', 'price']]
msci_china_df.rename(columns={'price': f'msci_china_price'}, inplace=True)
msci_china_df['date'] = pd.to_datetime(msci_china_df.date, format='%d/%m/%Y') # set the date based on yymmdd
msci_china_df = msci_china_df.set_index('date')
msci_china_df.dropna(inplace=True)

msci_china_df.sort_index(ascending=True, inplace=True)
msci_china_df = aggregate_period(msci_china_df, 'M', 'last')
msci_china_df = msci_china_df.apply(pd.to_numeric)
msci_china_df['msci_china_returns'] = msci_china_df['msci_china_price']/msci_china_df['msci_china_price'].shift(1) - 1
msci_china_df['msci_china_returns_lead_1'] = msci_china_df['msci_china_returns'].shift(-1)
msci_china_df

# import msci weights

def extract_ticker_name(bm_ticker_name):
    """
    Extract Ticker Name with SEDOL
    """
    bm3 = bm_ticker_name.copy()
    benchmark=bm3.iloc[6,].unique()
    benchmark=[x for x in benchmark if str(x) != 'nan']
    periods=bm3.iloc[5,].unique()
    periods=[pd.to_datetime(x).year*100+pd.to_datetime(x).month for x in periods if str(x) != 'nan']
    bm3=bm3[bm3['Unnamed: 2'].isna()==False].iloc[4:,:]
    bm3.index=bm3.index.astype('str')
    bm3=bm3[bm3.index!='--']

    df = bm3.iloc[:, :1]
    df.reset_index(inplace=True)
    df.columns = ['SEDOL', 'TICKER_NAME']
    
    return df


def bm():
    
    # Getting the Weights
    weight = pd.read_excel(r'unknown', index_col=0, header=6)
    weight['len']=[len(str(i)) for i in weight.index.values]
    weight = weight[weight['len']==7].drop(columns=['Unnamed: 1', 'len']).unstack().reset_index()
    weight.columns = ['Date', 'SEDOL', 'Weight']
    weight = weight[~weight['Weight'].isin(['--', 0])]
    weight['Date'] = pd.to_datetime(weight['Date'])
    weight['Year'] = weight['Date'].dt.year
    
    # Left Merge with daf ticker
    ticker=pd.read_excel('inputs/daf_ticker.xlsx',sheet_name='Sheet1').rename(columns={'CNTRY_OF_RISK':'MSCI COUNTRY','GICS_SECTOR_NAME':'GICS_SECTOR'})
    ticker['SEDOL']=ticker['SEDOL'].astype('str')
    ticker['CDR COUNTRY1']=[x[-9:-8] for x in ticker['Ticker']]
    ticker['CDR COUNTRY']=np.where(ticker['CDR COUNTRY1']=='C','CHINA',ticker['Ticker'])
    
    ticker = ticker.merge(weight, how='left', left_on='SEDOL', right_on='SEDOL')
    
    # filter for china market
    ticker = ticker[['SEDOL', 'Ticker', 'MSCI COUNTRY', 'GICS_SECTOR', 'CDR COUNTRY', 'Date', 'Weight']]
    
    return ticker

# new method
msci_weights_df = bm()
msci_weights_df = msci_weights_df[(msci_weights_df['MSCI COUNTRY']=="CN")]
financials_weights_df = msci_weights_df.copy()

# filter for HK equity
msci_weights_df = msci_weights_df[msci_weights_df['GICS_SECTOR']=="Financials"]
msci_weights_df = msci_weights_df[msci_weights_df['CDR COUNTRY'].str.contains(r'HK Equity')]
msci_weights_df.set_index('Date', inplace=True, drop=True)
msci_weights_df.index.names = ['date']
msci_weights_df.index = pd.to_datetime(msci_weights_df.index, format='%Y%m') # set the date based on yyyydd
msci_weights_df.index = msci_weights_df.index + MonthEnd(0) # convert to Month End
msci_weights_df = standardize_col_names(msci_weights_df)
msci_weights_df = msci_weights_df[msci_weights_df.index.notnull()] # remove null dates

msci_weights_df = msci_weights_df[['sedol', 'weight']]
msci_weights_df['weight'] = msci_weights_df['weight'].astype('float')
msci_weights_df = msci_weights_df.pivot_table(index='date', columns='sedol', values='weight')
msci_weights_df = standardize_col_names(msci_weights_df)
msci_weights_df.sort_index(ascending=True, inplace=True)
msci_weights_df = msci_weights_df.apply(pd.to_numeric) # convert the string objects to numeric
msci_weights_df.fillna(0, inplace=True)
msci_weights_df

## Consolidating Finalised Signal From Financial Industry Model
# Using ML with ESS Feature Engineering as final model (XGBoost Classifier), utilising monthly rebalancing operation.


# get msci financials sedol
financials_weights_df = financials_weights_df[financials_weights_df.Date.notnull()] # remove null dates
check_df = financials_weights_df[financials_weights_df['GICS_SECTOR']=="Financials"]
check_df.drop_duplicates(subset=['CDR COUNTRY', 'Ticker', 'SEDOL'], inplace=True)
financials_sedol_list = list(check_df.SEDOL.unique())
financials_sedol_list = [i.lower() for i in financials_sedol_list] # make it all small caps

# to get the overall weights of china
financials_weights_df.set_index('Date', inplace=True, drop=True)
financials_weights_df.index.names = ['date']
financials_weights_df.index = pd.to_datetime(financials_weights_df.index, format='%Y%m') # set the date based on yyyydd
financials_weights_df.index = financials_weights_df.index + MonthEnd(0) # convert to Month End
financials_weights_df = standardize_col_names(financials_weights_df)

financials_weights_df = financials_weights_df[['sedol', 'weight']]
financials_weights_df['weight'] = financials_weights_df['weight'].astype('float')

checking_list = financials_weights_df.sedol.unique()
financials_weights_df = financials_weights_df.pivot_table(index='date', columns='sedol', values='weight')
financials_weights_df = standardize_col_names(financials_weights_df)
financials_weights_df.sort_index(ascending=True, inplace=True)
financials_weights_df = financials_weights_df.apply(pd.to_numeric) # convert the string objects to numeric
financials_weights_df.fillna(0, inplace=True)
financials_weights_df = financials_weights_df.apply(lambda x: x/sum(x), axis=1) # scale the weights
financials_weights_df

len(financials_sedol_list)

# filter only financials companies
financials_weights_df = financials_weights_df[financials_sedol_list]
financials_weights_df['financials'] = financials_weights_df.sum(axis=1)
financials_weights_df['others'] = 1 - financials_weights_df['financials']
financials_weights_df

# import pickle for financials weights
from functools import reduce

dfs = [financials_weights_df[['financials', 'others']], msci_financial_df, msci_china_df]
fin_returns_df = reduce(lambda  left,right: pd.merge(left,right,left_index=True,right_index=True, how='left'), dfs)

fin_returns_df['others_returns'] = (fin_returns_df['msci_china_returns'] - fin_returns_df['msci_financial_returns']*fin_returns_df['financials'])/fin_returns_df['others']
fin_returns_df['others_returns_lead_1'] = fin_returns_df['others_returns'].shift(-1)
fin_returns_df['others_lead_1'] = fin_returns_df['others'].shift(-1)
fin_returns_df['financials_lead_1'] = fin_returns_df['financials'].shift(-1)
fin_returns_df = fin_returns_df.ffill(axis=0).bfill(axis=0)
fin_returns_df

# concat with actual financials weights - the ones with ravenpack
msci_weights_df['total_hk_fin_weights'] = msci_weights_df.sum(axis=1) # total
msci_weights_df = pd.concat([msci_weights_df, financials_weights_df[['others']]], axis=1)
msci_weights_df

# for training of optimal parameters - to be deleted
file = open('outputs/predict_train_dict.pickle', "rb")
predict_dict = pickle.load(file)

def rename_returns_col(df):
    """
    Rename the columns to based be based off banks, to allow easier concatenation of the df
    """
    
    company_name = df['sedol'][0].lower() # make it lower case
    
    rename_dict = {
        'returns_predict': f'{company_name}_returns_predict',
        'returns_predict_proba': f'{company_name}_returns_predict_proba',
        'position' : f'{company_name}_position',
        'event_sentiment_score': f'{company_name}_event_sentiment_score',
        'regulatory_event_sentiment_score': f'{company_name}_regulatory_event_sentiment_score',
        'product_event_sentiment_score': f'{company_name}_product_event_sentiment_score',
        'ratings_event_sentiment_score': f'{company_name}_ratings_event_sentiment_score',
        'news_spikes_m': f'{company_name}_news_spikes_m',
        'macd_12_26': f'{company_name}_macd_12_26'
    }
    df.rename(columns=rename_dict, inplace=True)
    df = standardize_col_names(df) # make to all lower caps
    df = df.drop(['sedol'], axis=1)
    
    return df

for sedol, df in predict_dict.items():
    
    df = rename_returns_col(df)
    predict_dict[sedol] = df # replacing back the dictionary

hk_financials_sedol_list = [i.lower() for i in list(predict_dict.keys())] # hk financials sedol, accounting for ravenpack news
predict_dict

# merge the dataframes with weights_df
from functools import reduce

dfs = list(predict_dict.values()) + [msci_financial_df, msci_china_df]
df = reduce(lambda  left,right: pd.merge(left,right,left_index=True,right_index=True, how='left'), dfs)
df = df.merge(msci_weights_df, left_index=True, right_index=True, how='left')
df

# to be deleted - for research & testing purpose
test_date = "2019-01-01"
train_df = df[df.index < test_date]
test_df = df[df.index >= test_date]
df = train_df

### Threshold Analysis
Conducting threshold analysis for financials to optimize parameters based on best excess return. Note, not a general quantile for financial sector, it is customise individually for each firms.

%time 
%run import_library.ipynb

best_performance_values, best_excess_return = optimize_financial_quantile(df, 'msci_financial', hk_financials_sedol_list, 0.90)
print(f"=== Best Excess Return based on tuning is : {best_excess_return}")

print(f"=== Best Performance Parameters: {best_performance_values} ====")
print(f"=== Best Excess Returns: {best_excess_return} ====")

best_excess_return

# best_performance_values = [0.8, 0.5, 2.0]

Previous Best values: [0.8, 0.6, 2.5], Best excess return, 0.256836
for the tuning: 0.9, 0.4, 3.5 --> bad excess return --> explanation: the past optimisation of threshold does not reflect well the future threshold.

df = test_df

# best performance values based on excess returns tuning
quantile = best_performance_values[0]
returns_threshold = best_performance_values[1]
news_spikes_threshold = best_performance_values[2]

new_df = prediction_processing_financials(df, hk_financials_sedol_list, True, quantile, returns_threshold, news_spikes_threshold)
new_df['sedol'] = 'msci_financials' # hardcoded as of date
new_df

# performance breakdown
%run import_library.ipynb

performance_financial = performance_breakdown(new_df, 'msci_financials', 'M')
performance_breakdown_df = pd.DataFrame(performance_financial,index=['Annualised Return (Buy Hold)','Annualised Return (Sentiment)','Annualised Excess Return', 'Winrate', "Annualised Turnover", 'Volatility (Buy Hold)', 'Volatility (Sentiment)', 'Max Drawdown (Buy Hold)', 'Max Drawdown (Sentiment)',"Cumulative Return (Buy Hold)", "Cumulative Return (Sentiment)", 'Sharpe Ratio', 'F1 Score', "Precision"])

# # formatting
performance_breakdown_financial_df = perf_format(performance_breakdown_df).copy()
performance_breakdown_financial_df

new_df.to_clipboard()

def stop

performance_financial = performance(new_df, 'msci_financials', 'M')
performance_df = pd.DataFrame(performance_financial,index=['Annualised Return (Buy Hold)','Annualised Return (Sentiment)','Annualised Excess Return', 'Winrate', "Annualised Turnover", 'Volatility (Buy Hold)', 'Volatility (Sentiment)', 'Max Drawdown (Buy Hold)', 'Max Drawdown (Sentiment)',"Cumulative Return (Buy Hold)", "Cumulative Return (Sentiment)", 'Sharpe Ratio', 'F1 Score', "Precision"], columns=['MSCI Financials'])

# formatting
performance_overview_financial_df = perf_format(performance_df).copy()
performance_overview_financial_df

import plotly.graph_objects as go
import numpy as np

fig_financial_abs_returns = go.Figure()

# Add traces
fig_financial_abs_returns.add_trace(go.Scatter(x=new_df.index, y=new_df['return_mp'], name="MSCI Financials Sentiment Strategy"))
fig_financial_abs_returns.add_trace(go.Scatter(x=new_df.index, y=new_df['return_bm'], name="MSCI Financials Buy Hold Strategy"))
fig_financial_abs_returns.update_layout(
    title="MSCI Financials Returns",
    xaxis_title="Time Period",
    yaxis_title="Returns (Arithmetic)"
)

import plotly.graph_objects as go
import numpy as np

fig_financial_cum_returns = go.Figure()

# Add traces
fig_financial_cum_returns.add_trace(go.Scatter(x=new_df.index, y=new_df['return_mp_cum'], name="MSCI Financials Sentiment Strategy"))
fig_financial_cum_returns.add_trace(go.Scatter(x=new_df.index, y=new_df['return_bm_cum'], name="MSCI Financials Buy Hold Strategy"))
fig_financial_cum_returns.update_layout(
    title="MSCI Financials Cumulative Returns",
    xaxis_title="Time Period",
    yaxis_title="Cumulative Returns (Arithmetic)"
)

## Visualisation of Others Return
Comparing Financials Returns vs Others Returns

import plotly.graph_objects as go
import numpy as np

fin_returns_df = fin_returns_df[fin_returns_df.index >= new_df.index[0]]
fig_cn_others_abs_returns = go.Figure()

# Add traces
fig_cn_others_abs_returns.add_trace(go.Scatter(x=fin_returns_df.index, y=fin_returns_df['msci_financial_returns'], name="MSCI Financial Returns"))
fig_cn_others_abs_returns.add_trace(go.Scatter(x=fin_returns_df.index, y=fin_returns_df['others_returns'], name="MSCI Others Returns"))
fig_cn_others_abs_returns.update_layout(
    title="MSCI Financial vs Others Returns Combined",
    xaxis_title="Time Period",
    yaxis_title="Returns"
)

## MSCI Portfolio Allocation
In this script, we will be backtesting on the monthly dataset, allocating weights based on financials and non-financials data in MSCI Singapore, identifying if the overweight and underweight trading strategies can outperform the benchmark (MSCI Singapore).

import copy
msci_portfolio_weights_df = new_df.copy()

# filtering only relevant columns for weight allocation for msci financials
filter_cols = [
    'position',
    'position_lag_1',
    'msci_china_price',
    'msci_china_returns',
    'msci_china_returns_lead_1', # for performance measure
    'msci_financial_price',
    'msci_financial_returns',
    'msci_financial_returns_lead_1',
    "tot_fin_weights",
    "others",
    "returns_predict", 
]

msci_portfolio_weights_df = msci_portfolio_weights_df[filter_cols]
msci_portfolio_weights_df['msci_other_returns'] = ((msci_portfolio_weights_df['msci_china_returns'] - msci_portfolio_weights_df['msci_financial_returns']* msci_portfolio_weights_df['tot_fin_weights']/100) / (msci_portfolio_weights_df['others']/100))/100
msci_portfolio_weights_df

optimal_weight, _ = optimize_weight_distribution(msci_portfolio_weights_df, 0.1)
optimal_weight

msci_portfolio_weights_df = weight_allocation(msci_portfolio_weights_df, optimal_weight) # allocate weights
performance_financial = performance(msci_portfolio_weights_df, 'msci_china', 'M', True) # set overweight underweight to true
performance_df = pd.DataFrame(performance_financial,index=['Annualised Return (Benchmark)','Annualised Return (Sentiment)','Annualised Excess Return', 'Winrate', "Annualised Turnover", 'Volatility (Benchmark)', 'Volatility (Sentiment)', 'Max Drawdown (Benchmark)', 'Max Drawdown (Sentiment)',"Cumulative Return (Benchmark)", "Cumulative Return (Sentiment)", 'Sharpe Ratio', 'F1 Score', "Precision"], columns=['MSCI China'])

# formatting
performance_overview_cn_df = perf_format(performance_df, True).copy() # set benchmark as True
performance_overview_cn_df

### Performance Breakdown by Year (Test Dataset)

# performance breakdown
performance_cn = performance_breakdown(msci_portfolio_weights_df, 'msci_china', 'M', True) # set overweight underweight to true
performance_breakdown_df = pd.DataFrame(performance_cn,index=['Annualised Return (Buy Hold)','Annualised Return (Sentiment)','Annualised Excess Return', 'Winrate', "Annualised Turnover", 'Volatility (Buy Hold)', 'Volatility (Sentiment)', 'Max Drawdown (Buy Hold)', 'Max Drawdown (Sentiment)',"Cumulative Return (Buy Hold)", "Cumulative Return (Sentiment)", 'Sharpe Ratio', 'F1 Score', "Precision"])
performance_breakdown_df.rename(index={"Annualised Return (Buy Hold)": "Annualised Return (Benchmark)", "Volatility (Buy Hold)": "Volatility (Benchmark)", "Max Drawdown (Buy Hold)": "Max Drawdown (Benchmark)", "Cumulative Return (Benchmark)": "Cumulative Return (Benchmark)"}, inplace=True)

# formatting
performance_breakdown_cn_df = perf_format(performance_breakdown_df, True).copy()
performance_breakdown_cn_df

### Visualisation of Fund Performance

# ideal strategy - for cumulative return

def ideal_weight_allocation(df, weight=0.05):
    """
    Assign weights allocation based on threshold, add in additional weights based on model signals
    Current: 0.05 +/-
    """
    
    df['return_bm'] = df['msci_china_returns']+1
    df['return_bm_cum'] = (df['return_bm']).cumprod()
    
    # The following is for 100% accurate forecasting, ie, knowing the future (just for presentation)
    df['return_ideal_mp'] = df.apply(lambda x: x['msci_financial_returns'] * (x['tot_fin_weights'] + weight) \
    + x['msci_other_returns'] * (x['others'] - weight) if x['msci_financial_returns'] > x['msci_other_returns'] \
    else x['msci_financial_returns'] * (x['tot_fin_weights'] - weight) \
    + x['msci_other_returns'] * (x['others'] + weight), axis=1)+1
    df['position'] = df.apply(lambda x: 1 if x['msci_financial_returns'] > x['msci_other_returns'] else 0, axis=1)
    
    df['return_ideal_cum'] = (df['return_ideal_mp']).cumprod()
    
    # excess return mp for winrate calculation
    df['excess_return_mp'] = df['return_ideal_mp'] - df['return_bm']

    # creating strategy directions
    df['returns_movement'] = df.apply(lambda x: msci_china_returns_movement(x), axis=1)
    df['sedol'] = 'msci_china'                      
    
    return df

ideal_df = ideal_weight_allocation(msci_portfolio_weights_df.copy(), optimal_weight) # allocate weights

import plotly.graph_objects as go
import numpy as np

fig_cn_abs_returns = go.Figure()

# Add traces
fig_cn_abs_returns.add_trace(go.Scatter(x=msci_portfolio_weights_df.index, y=msci_portfolio_weights_df['return_mp'], name="MSCI China Sentiment Strategy"))
fig_cn_abs_returns.add_trace(go.Scatter(x=msci_portfolio_weights_df.index, y=msci_portfolio_weights_df['return_bm'], name="MSCI China Benchmark Strategy"))
fig_cn_abs_returns.update_layout(
    title="MSCI China Returns",
    xaxis_title="Time Period",
    yaxis_title="Returns"
)

import plotly.graph_objects as go
import numpy as np

fig_cn_cum_returns = go.Figure()

# Add traces
fig_cn_cum_returns.add_trace(go.Scatter(x=msci_portfolio_weights_df.index, y=msci_portfolio_weights_df['return_mp_cum'], name="MSCI China Sentiment Strategy"))
fig_cn_cum_returns.add_trace(go.Scatter(x=msci_portfolio_weights_df.index, y=msci_portfolio_weights_df['return_bm_cum'], name="MSCI China Benchmark Strategy"))
fig_cn_cum_returns.add_trace(go.Scatter(x=ideal_df.index, y=ideal_df['return_ideal_cum'], name="MSCI China Ideal Strategy"))
fig_cn_cum_returns.update_layout(
    title="MSCI China Cumulative Returns",
    xaxis_title="Time Period",
    yaxis_title="Cumulative Returns"
)

### Position for Next Month

position_df = msci_portfolio_weights_df.tail(12)[['position']].copy()
position_df

import plotly.graph_objects as go
import numpy as np

fig_position_cn_returns = go.Figure()

# Add traces
fig_position_cn_returns.add_trace(go.Scatter(x=position_df.index, y=position_df['position'], name="MSCI China Trading Positions"))
fig_position_cn_returns.update_layout(
    title="MSCI China Trading Positions",
    xaxis_title="Time Period",
    yaxis_title="Trading Position"
)

import plotly.graph_objects as go
import numpy as np

position_df_extended = msci_portfolio_weights_df[['position']].copy()
fig_position_cn_returns_extended = go.Figure()

# Add traces
fig_position_cn_returns_extended.add_trace(go.Scatter(x=position_df_extended.index, y=position_df_extended['position'], name="MSCI China Trading Positions"))
fig_position_cn_returns_extended.update_layout(
    title="MSCI China Trading Positions",
    xaxis_title="Time Period",
    yaxis_title="Trading Position"
)

import plotly.graph_objects as go
import numpy as np

position_df_extended_combined = msci_portfolio_weights_df[['position']].copy()
position_df_extended_combined['position'] = position_df_extended_combined['position'].map({1:0.2, 0:-0.2}, na_action=None)

fin_returns_df = fin_returns_df[fin_returns_df.index >= new_df.index[0]]
fig_cn_others_abs_returns_combined = go.Figure()

# Add traces
fig_cn_others_abs_returns_combined.add_trace(go.Scatter(x=fin_returns_df.index, y=fin_returns_df['msci_financial_returns'], name="MSCI Financial Returns"))
fig_cn_others_abs_returns_combined.add_trace(go.Scatter(x=fin_returns_df.index, y=fin_returns_df['others_returns'], name="MSCI Others Returns"))
fig_cn_others_abs_returns_combined.add_trace(go.Scatter(x=position_df_extended_combined.index, y=position_df_extended_combined['position'], name="MSCI China Trading Positions"))
fig_cn_others_abs_returns_combined.update_layout(
    title="MSCI Financial vs Others Returns Combined",
    xaxis_title="Time Period",
    yaxis_title="Returns"
)

### Dashboard For Reports

%matplotlib inline
import matplotlib.pyplot as plt
import pandas as pd
import ipywidgets as widgets
import numpy as np

out1 = widgets.Output()
out2 = widgets.Output()
out3 = widgets.Output()
out4 = widgets.Output()
out5 = widgets.Output()

tab = widgets.Tab(children = [out1, out2, out3, out4, out5])
tab.set_title(0, 'Model Outputs')
tab.set_title(1, 'MSCI China')
tab.set_title(2, 'MSCI Financials')
tab.set_title(3, 'MSCI Others')
tab.set_title(4, 'Detailed Breakdown')
display(tab)

with out1:
    temp_position_df = position_df.copy().style.set_caption("Position For Financials Sector (From Past 12 Months)")
    display(temp_position_df, fig_position_cn_returns)

with out2:
    temp_performance_overview_cn_df = performance_overview_cn_df.copy().style.set_caption("MSCI China OW/UW Strategy Performance Metrics")
    display(temp_performance_overview_cn_df, fig_cn_abs_returns, fig_cn_cum_returns)

with out3:
    temp_performance_overview_financial_df = performance_overview_financial_df.copy().style.set_caption("MSCI Financials Buy Hold Strategy Performance Metrics")
    display(temp_performance_overview_financial_df, fig_financial_abs_returns, fig_financial_cum_returns)
    
with out4:
    display(fig_cn_others_abs_returns_combined, fig_cn_others_abs_returns, fig_position_cn_returns_extended, position_df_extended)
    
with out5:
    temp_performance_breakdown_cn_df = performance_breakdown_cn_df.copy().style.set_caption("MSCI China OW/UW Strategy")
    temp_performance_breakdown_financial_df = performance_breakdown_financial_df.copy().style.set_caption("MSCI Financials Buy Hold Strategy")
    display(temp_performance_breakdown_cn_df, temp_performance_breakdown_financial_df)